In [1]:
exec(open("../header.py", "r").read())

In [4]:
from sklearn.svm import SVC
from sklearn import datasets

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [5]:
data = datasets.fetch_20newsgroups(subset="all",
                                   categories=['alt.atheism', 'sci.space'])

In [6]:
vectorizer = TfidfVectorizer()

In [7]:
vectorized_X = vectorizer.fit_transform(data.data)

In [9]:
grid = {"C": 10**sp.arange(-5,6,1,dtype=sp.float64)}
clsfr = SVC(kernel="linear", random_state=241)
cross_validator = KFold(n_splits=5, shuffle=True, random_state=241)
grid_search = GridSearchCV(clsfr, grid, scoring="accuracy", cv=cross_validator)

In [10]:
grid_search.fit(vectorized_X, data.target)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [11]:
scores = [(param["C"], score) for param, score in zip(grid_search.cv_results_["params"], grid_search.cv_results_["mean_test_score"]) ]
scores = sp.array(scores)
display(scores)

best_C = scores[sp.argmax(scores[:, 1]), 0]

array([[1.00000000e-05, 5.52631579e-01],
       [1.00000000e-04, 5.52631579e-01],
       [1.00000000e-03, 5.52631579e-01],
       [1.00000000e-02, 5.52631579e-01],
       [1.00000000e-01, 9.50167973e-01],
       [1.00000000e+00, 9.93281075e-01],
       [1.00000000e+01, 9.93281075e-01],
       [1.00000000e+02, 9.93281075e-01],
       [1.00000000e+03, 9.93281075e-01],
       [1.00000000e+04, 9.93281075e-01],
       [1.00000000e+05, 9.93281075e-01]])

In [12]:
best_svm = SVC(kernel="linear", random_state=241, C=best_C)

In [13]:
best_svm.fit(vectorized_X, data.target)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

In [30]:
@answer
def valuable_words(fout):
    ids = sp.argsort(sp.absolute(best_svm.coef_.toarray()[0]))[-10:]
    feature_names = vectorizer.get_feature_names()
    words = [feature_names[i] for i in ids]
    words = sorted(words, key=str.lower)
    fout.write(",".join(words))
valuable_words()

*** OUTPUT (output/01_valuable_words.dat) ***
atheism,atheists,bible,god,keith,moon,religion,sci,sky,space
